In [1]:
from mido import Message
from mido import MidiFile
from pymidifile import *
from reformat_midi import *
import numpy as np
import os

In [2]:
def GetTrackName(track):
    """Extract the track name in a track.
    """
    for msg in track:
        if (msg.type == 'track_name'):
            return msg.name
    
    
def FromMIDIToChromosome(filename):
    """Convert a MIDI file into chromosome representation
    Parameters:
        filename: Path of the MIDI file. The file should be single-track and properly quantized.
    
    Return value:
        a list of numbers in the range 0..28, viz. the chromosome representation. 
        (Pitches beyond the range 1..27 are discarded.)
    [Note] Integer representation of pitches in Chromosome:
        0 = rest,
        1 = F3, 2 = #F3, ... , 26 = #F5, 27 = G5
        28 = duration extension
        unit duration = an eighth note
    [Note] Integer representation of pitches as MIDI note number:
        60 = C4
        53 = F3, 54 = #F3, ... , 78 = #F5, 79 = G5
    [Note] Rule of convertion:
        Chromosome number + 52 == MIDI note number
    
    Notes
    """
    
    #mid = MidiFile(filename)
    #print(mid.ticks_per_beat)
    #for i,track in enumerate(mid.tracks):
    #    print(i,GetTrackName(track))
    #x = int(input())
    #for msg in mid.tracks[0]:
    #    print(msg)
    reformatted = reformat_midi(filename, verbose=False, write_to_file=False, override_time_info=True)
    matrix = mid_to_matrix(reformatted)
    quantizer = quantize_matrix(matrix, stepSize=0.5, quantizeOffsets=True, quantizeDurations=True)
    note_list = np.array(sorted(quantizer, key=lambda x: x[0], reverse=False))
    #print(note_list[:,1]) # unit duration = quarter note
    #print(note_list[:,2])
    #print(note_list[:,1]+note_list[:,2])
    #print(2*np.max(note_list[:,1]+note_list[:,2]))
    length = int(2*np.max(note_list[:,1]+note_list[:,2]))
    output = np.zeros(length)
    #for note in note_list:
    for note in sorted(quantizer, key=lambda x: x[0], reverse=False):
        #print(note)
        start = int(2*note[1])
        dur = int(2*note[2])
        pitch = int(note[0] - 52)
        if (pitch in range(1,28)): # if pitch < 1 (F3) or pitch > 27 (G5), it is neglected (replaced with rests)
            for t in range(start, start+dur):
                output[t] = pitch
    #for i in range(length-1, 0, -1):
        #if (output[i] == output[i-1]):
            #output[i] = 28
    output_list = output.tolist()
    #print(output.tolist())
    while (len(output_list) >= 32):
        #for i in range(31, 0, -1):
            #if (output_list[i] == output_list[i-1]):
                #output_list[i] = 28
        #print(output_list[:32])
        
        cnt = 0
        for i in range(0, 32):
            if (int(output_list[i]) == 0):
                cnt += 1
        #print(cnt)
        if (cnt >= 8):
            output_list = output_list[32:]
            continue
        
        print(0, end = ' ')
        for i in range(0, 32):
            print(int(output_list[i]), end = ' ')
        print('')
        output_list = output_list[32:]
    #return output.tolist()

In [3]:
def FromMIDIFolderToChromosome(foldername):
    for root, dirs, files in os.walk(foldername):
        for name in files:
            FromMIDIToChromosome(os.path.join(root, name))

In [4]:
FromMIDIToChromosome('chopin.mid')

0 19 19 19 14 20 19 17 15 14 14 15 15 14 0 0 0 19 19 19 19 19 19 21 21 22 22 22 22 22 22 22 22 
0 21 21 21 21 21 21 21 21 19 19 11 11 12 12 14 14 27 27 27 27 27 27 26 24 23 23 23 23 23 26 26 27 
0 19 19 19 19 19 19 19 17 14 14 15 24 23 23 23 23 22 22 22 22 22 22 20 20 13 19 19 15 15 15 15 15 
0 17 17 17 17 17 17 17 17 19 19 19 19 19 19 21 21 22 22 22 22 21 21 21 21 23 23 23 23 24 24 19 19 
0 18 18 18 18 19 19 19 19 19 19 19 19 18 18 18 18 19 19 19 19 0 0 0 0 23 23 23 23 21 21 21 0 
0 19 19 19 19 19 19 20 20 21 21 21 23 24 26 27 27 27 27 26 26 26 26 26 26 23 23 23 23 21 21 21 19 
0 21 21 19 19 19 19 19 19 18 18 18 18 15 14 15 15 17 17 17 17 17 17 0 0 27 27 27 27 27 27 20 19 
0 23 23 23 23 0 26 27 17 19 19 19 19 19 19 19 17 14 14 15 24 23 23 23 23 23 23 23 23 23 23 23 23 
0 24 24 24 24 24 24 24 24 27 27 27 26 24 24 23 23 24 24 24 24 24 24 0 0 24 24 24 24 26 26 26 26 
0 24 24 24 24 24 24 25 25 26 26 26 24 26 0 23 23 24 24 24 24 0 0 0 0 24 24 16 24 22 26 16 26 
0 26 26 24 24 0 0 0 0 23 23 

In [5]:
FromMIDIFolderToChromosome('./data/chopin', )

0 0 0 0 0 0 0 7 7 16 16 16 17 19 23 24 21 19 19 19 19 14 14 14 16 12 12 12 12 14 16 17 19 
0 21 21 21 21 0 21 21 19 19 19 17 17 0 16 16 14 13 13 14 14 0 6 7 17 16 16 11 12 11 12 12 14 
0 12 12 12 12 11 23 21 19 16 16 16 17 24 26 26 21 21 19 19 19 14 14 14 16 12 12 11 12 14 16 17 19 
0 21 21 21 21 0 21 21 19 21 19 18 18 18 19 21 22 23 23 23 21 18 19 9 0 0 0 14 14 14 19 23 23 
0 18 18 18 18 19 19 0 0 16 16 16 17 19 21 24 22 13 13 13 13 14 14 0 0 18 18 18 19 21 23 26 24 
0 15 15 15 15 16 17 21 19 13 13 13 13 14 15 19 17 17 17 15 14 11 11 11 11 27 27 24 20 8 5 15 6 
0 7 7 7 7 9 11 12 14 16 16 16 17 26 0 0 26 21 19 19 19 14 14 14 15 12 12 12 13 15 19 20 17 
0 15 15 15 15 10 10 10 12 9 9 9 9 9 11 12 13 14 14 7 7 16 16 16 16 0 16 21 26 26 16 16 0 
0 12 12 0 0 0 0 12 12 16 16 16 16 16 16 12 14 16 16 16 16 14 14 12 12 21 21 19 19 12 12 14 16 
0 17 16 0 0 0 12 24 24 24 24 26 24 16 14 12 14 15 16 0 12 24 23 16 12 24 23 11 17 20 23 26 23 
0 17 12 12 8 11 11 4 8 11 12 12 16 14 16 16 19 19 16 16 11 

0 0 0 0 23 23 23 23 23 24 24 23 14 21 21 21 23 23 23 23 19 19 19 19 19 23 23 14 14 26 26 26 26 
0 26 26 24 24 0 23 23 23 26 24 24 23 21 21 21 21 23 23 23 18 0 19 21 21 19 19 23 14 14 14 26 26 
0 26 26 26 26 24 24 23 23 23 22 22 22 23 25 0 0 4 26 26 26 26 25 25 23 23 23 23 23 18 18 18 21 
0 23 23 21 0 19 19 19 19 19 13 2 0 21 21 21 23 26 27 24 2 23 21 21 19 19 19 21 21 21 21 18 18 
0 0 19 19 21 21 21 23 23 26 24 24 23 21 0 19 7 0 21 21 21 21 21 21 21 14 12 23 23 23 23 23 21 
0 21 14 14 22 22 22 22 12 12 12 12 11 0 11 19 23 23 23 23 23 24 24 23 21 21 21 21 23 23 23 23 19 
0 19 19 19 19 19 23 23 2 2 26 26 26 26 26 26 24 24 23 23 23 26 24 24 23 21 21 21 21 14 23 23 23 
0 19 21 19 23 26 2 0 14 14 9 9 14 14 2 2 24 24 23 23 23 23 22 11 23 25 25 4 4 26 26 26 10 
0 19 19 21 18 12 19 23 23 23 0 23 23 23 23 23 23 23 23 21 21 19 19 19 19 19 19 19 19 11 26 26 26 
0 26 24 24 23 23 23 21 21 21 21 6 21 0 23 21 19 18 0 19 19 13 19 19 9 0 19 19 19 18 0 0 0 
0 23 21 20 21 21 0 26 26 0 21 21 19 18 19 0 0 

0 0 20 20 25 24 22 25 27 26 22 19 19 19 0 17 17 18 18 0 0 20 20 21 26 26 25 23 26 18 18 18 0 
0 26 25 26 26 0 26 26 25 26 25 26 26 25 26 25 25 25 25 26 23 20 20 20 18 20 17 13 13 0 25 27 24 
0 20 20 0 19 20 22 24 22 25 27 25 22 24 0 0 19 20 22 24 22 25 27 25 22 24 0 0 19 20 22 17 0 
0 13 16 12 0 0 0 11 14 15 15 15 15 15 17 15 15 7 15 15 17 15 15 13 15 15 17 15 15 12 15 15 24 
0 24 24 22 20 20 17 15 15 15 15 15 17 15 15 13 15 15 17 15 15 12 15 15 20 20 20 19 17 17 12 19 19 
0 19 19 19 21 19 19 17 19 19 21 19 19 16 19 19 16 17 17 26 24 24 21 19 19 19 19 19 21 19 19 17 19 
0 19 21 19 19 16 19 19 21 22 22 17 22 22 21 21 21 21 21 21 22 22 22 21 21 21 26 26 26 25 25 25 22 
0 22 22 21 21 21 22 23 23 23 23 23 24 24 24 23 23 23 19 21 21 27 27 27 26 26 26 25 25 25 24 25 25 
0 25 13 13 15 13 13 11 13 13 15 13 13 10 13 13 22 22 22 20 18 18 15 13 13 13 13 13 15 13 13 11 13 
0 13 15 13 13 10 13 13 18 18 18 17 15 15 10 17 17 17 17 17 19 17 17 15 17 17 19 17 17 14 17 17 26 
0 26 26 24 22 22 19 17 17 1

0 27 19 19 19 20 20 20 20 20 22 22 22 23 27 27 25 22 22 22 22 22 22 23 23 23 24 24 24 25 25 25 23 
0 23 22 20 20 20 22 22 22 23 23 23 23 23 23 18 18 18 20 20 20 22 22 22 20 20 18 17 17 17 17 17 17 
0 27 19 19 19 20 20 20 22 22 22 23 23 23 15 20 27 27 22 22 22 22 22 23 23 23 23 23 11 21 21 21 21 
0 21 21 21 21 21 20 20 19 20 20 25 23 23 23 21 21 21 20 20 19 20 20 20 23 23 23 23 23 23 22 22 21 
0 0 0 0 0 27 27 27 27 27 27 22 22 22 27 27 27 23 23 23 22 22 20 23 23 23 23 23 23 22 22 22 25 
0 25 23 22 22 22 17 17 22 18 18 18 17 17 15 27 27 27 27 27 27 27 27 27 22 22 23 25 25 25 27 27 22 
0 0 23 23 22 21 21 21 23 23 25 20 20 20 21 21 21 19 19 19 20 20 22 23 23 23 22 22 23 27 27 25 27 
0 0 19 20 20 20 23 23 22 19 19 19 20 20 22 23 23 23 25 25 27 18 16 23 23 23 27 26 26 26 27 27 27 
0 26 26 26 26 26 25 24 24 24 25 25 25 24 24 24 24 24 23 22 22 22 20 20 22 23 23 23 25 25 25 27 27 
0 22 22 23 23 23 23 23 27 25 22 22 22 22 22 23 23 23 23 23 11 21 21 21 21 21 21 21 21 21 20 20 19 
0 20 20 25 23 23

0 17 17 17 17 18 18 18 0 25 26 25 25 23 25 25 26 26 0 26 23 23 26 27 27 0 25 25 25 25 26 26 23 
0 25 20 23 21 21 26 25 25 25 26 26 25 25 25 23 21 21 22 22 22 26 0 26 26 26 26 25 26 26 23 21 24 
0 26 26 26 26 26 26 25 0 26 26 26 26 26 17 16 16 26 21 20 20 16 19 17 24 23 17 16 16 26 26 23 0 
0 16 19 17 24 23 0 0 23 23 23 23 23 23 22 22 22 21 21 0 21 21 21 21 20 18 16 20 23 25 25 25 23 
0 21 21 16 16 16 16 21 25 0 0 0 0 25 21 17 17 17 17 17 17 18 18 18 0 25 26 25 25 23 23 24 25 
0 26 0 26 23 24 26 27 0 0 25 25 25 24 26 21 24 22 24 22 21 21 26 24 24 24 24 26 24 24 24 22 21 
0 21 21 21 21 21 21 21 26 24 26 26 24 24 26 19 23 26 26 26 26 24 23 23 23 26 0 26 26 26 26 0 0 
0 0 26 26 26 0 0 0 26 26 26 0 26 26 26 26 0 26 23 0 27 26 25 25 23 21 21 16 16 16 16 21 25 
0 21 19 18 0 26 0 26 26 26 26 25 26 26 23 21 24 26 0 26 26 26 26 25 26 0 0 26 23 26 26 25 23 
0 21 13 16 16 16 18 18 18 11 11 23 0 0 23 21 11 11 16 16 16 9 9 0 0 27 27 0 26 26 0 25 25 
0 0 27 26 25 21 21 21 26 27 21 21 23 26 27 26 25 2

0 0 0 0 0 0 0 20 20 20 20 25 25 23 23 21 21 20 20 20 21 20 20 20 20 20 20 25 25 23 23 21 21 
0 20 20 20 23 20 20 20 20 20 20 25 25 23 23 21 21 20 20 20 20 20 20 20 21 16 16 16 6 16 16 15 15 
0 13 13 13 13 20 20 20 20 20 20 25 25 23 23 21 21 20 20 20 21 20 20 20 20 20 20 25 25 23 23 21 21 
0 20 20 20 23 20 20 20 20 20 20 25 25 23 23 21 21 20 20 20 18 20 20 20 23 21 16 16 6 16 16 15 15 
0 13 13 13 13 16 16 16 16 18 18 10 11 13 11 16 18 20 20 20 20 20 20 20 20 22 22 14 15 16 16 20 22 
0 24 24 24 24 25 25 23 23 21 21 20 20 21 21 20 20 18 18 16 16 15 15 18 16 15 15 15 15 15 15 18 16 
0 15 15 20 20 20 20 20 20 20 20 25 25 23 23 21 21 20 20 20 21 20 20 20 20 20 20 24 27 23 18 20 23 
0 20 11 16 21 20 20 20 20 20 20 25 25 23 23 21 21 20 20 20 18 20 20 20 20 21 16 16 6 16 16 15 15 
0 13 13 13 13 16 16 16 16 18 18 10 11 13 11 16 18 20 20 20 20 20 20 20 20 22 22 14 15 16 16 20 22 
0 24 24 24 24 25 25 23 21 21 21 20 20 21 21 20 20 18 18 16 16 15 15 18 16 15 15 15 15 15 15 18 16 
0 15 15 20 20 20 20